In [ ]:
import torch

from model import run_experiment

# Set random seed
torch.manual_seed(123)

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
run_experiment(
    functions=["chirp"],
    given_data=[[2], [2, 3]],
    noise_percent=[0, 5, 10, 15, 20],
    initialization_error=[10, 15, 20, 25, 30] + [-10, -15, -20, -25, -30],
    run_name="noisy_data_NEW",
    device=device,
    max_iterations=20_000,
    learning_rate=1e-3,
)